In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
import sys
import os
prefix = '/content/gdrive/My Drive/'
# modify "customized_path_to_your_homework" here to where you uploaded your homework
customized_path_to_your_homework = 'IDLSProject-main'
sys_path = os.path.join(prefix, customized_path_to_your_homework)
sys.path.append(sys_path)

In [ ]:
%cd '/content/gdrive/My Drive/IDLSProject-main'

In [1]:
import pandas as pd
import os
import pickle

data_dir = './data/pytorch'
with open(os.path.join(data_dir, 'word_dict_amazon.pkl'), "rb") as f:
    word_dict = pickle.load(f)

train = pd.read_csv(os.path.join(data_dir, 'train_amazon.csv'), header=None, names=None)
test_sample = pd.read_csv(os.path.join(data_dir, 'test_amazon.csv'), header=None, names=None)

In [2]:
from sklearn.model_selection import train_test_split
test, val = train_test_split(test_sample, test_size=0.5)
train.shape, test.shape, val.shape

((83000, 502), (10987, 502), (10988, 502))

In [3]:
import torch
import torch.utils.data

# Turn the input pandas dataframe into tensors
train_y = torch.from_numpy(train[[0]].values).float()
train_X = torch.from_numpy(train.drop([0, 1], axis=1).values).long()

# Build the dataset
train_ds = torch.utils.data.TensorDataset(train_X, train_y)
# Build the dataloader
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=50)

######val data
# Turn the input pandas dataframe into tensors
val_y = torch.from_numpy(val[[0]].values).float()
val_X = torch.from_numpy(val.drop([0, 1], axis=1).values).long()

# Build the dataset
val_ds = torch.utils.data.TensorDataset(val_X, val_y)
# Build the dataloader
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=50)


#### Test data
# Turn the input pandas dataframe into tensors
test_y = torch.from_numpy(test[[0]].values).float()
test_X = torch.from_numpy(test.drop([0, 1], axis=1).values).long()

# Build the dataset
test_ds = torch.utils.data.TensorDataset(test_X, test_y)
# Build the dataloader
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=50)
print(test_y.shape)

torch.Size([10987, 1])


In [4]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self,trial,vocab_size,embed_size,filter_size,kernel_size,dropout,seq_len):
        super(CNN, self).__init__()
        filter_size = trial.suggest_int("filter_size",32,256)
        dropout = trial.suggest_uniform("dropout",0.1, 0.6)
        embed_size = trial.suggest_int("embed_size",32,100)
        self.embedding = torch.nn.Embedding(vocab_size,embed_size)
        self.conv1 = torch.nn.Conv2d(1,filter_size,kernel_size=[kernel_size[0],embed_size])
        self.conv2 = torch.nn.Conv2d(1,filter_size,kernel_size=[kernel_size[1],embed_size])
        self.conv3 = torch.nn.Conv2d(1,filter_size,kernel_size=[kernel_size[2],embed_size])
        self.mp1 = torch.nn.MaxPool1d(seq_len+1-kernel_size[0])
        self.mp2 = torch.nn.MaxPool1d(seq_len+1-kernel_size[1])
        self.mp3 = torch.nn.MaxPool1d(seq_len+1-kernel_size[2])
        self.dense = torch.nn.Linear(filter_size*3,1)
        self.dropout = torch.nn.Dropout(dropout)
    
    def forward(self, x, target):
        embed_input = self.embedding(x)
        embed_input.unsqueeze_(1)
        x1 = torch.tanh(self.dropout(self.conv1(embed_input))).squeeze(3)
        x2 = torch.tanh(self.dropout(self.conv2(embed_input))).squeeze(3)
        x3 = torch.tanh(self.dropout(self.conv3(embed_input))).squeeze(3)
        f1 = self.mp1(x1).squeeze(2)
        f2 = self.mp2(x2).squeeze(2)
        f3 = self.mp3(x3).squeeze(2)
        hidden = torch.cat([f1,f2,f3], dim=1)
        logits = self.dense(hidden)
        prediction = torch.sigmoid(logits)
        target = target.view([-1,1])
        correct_pred = torch.eq(torch.round(prediction).type(target.type()),target)
        accuracy = torch.sum(correct_pred)
        return prediction, accuracy

In [10]:
filename = ""
def write_to_csv(trail_num, epochs, train_loss, train_acc, val_loss, val_acc, time_train):
    global filename
    filename = "./cnn-amazon-hyperband-trails/"+str(trail_num)+".csv"
    epoch = [i for i in range(epochs)]
    df_metrics = pd.DataFrame(list(zip(epoch, train_loss, train_acc, val_loss, val_acc, time_train)), columns =['Epoch', 'train_loss', 'train_acc', 'val_loss', 'val_acc', 'train_time'])
    df_metrics.to_csv(filename, index=False)    
    
def append_to_csv(epochs, accuracy):
    acc = [accuracy for i in range(epochs)]
    df_csv = pd.read_csv(filename)
    df_csv['Test_Accuracy']  = acc
    df_csv.to_csv(filename, index=False)

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 10
batch_size = 50
learning_rate = 0.001
embed_size = 300
seq_len = 500
dropout = 0.5
filter_size = 100
vocab_size = 10000
embed_dims = 32
kernel_size = [3,4,5]

In [12]:
import time
import numpy as np
import torch.optim as optim

def objective(trial):
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    #epochs = trial.suggest_int("epochs",5,15)
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-0)
    #momentum = trial.suggest_float("momentum", 0.0, 1.0)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = CNN(trial,vocab_size, embed_dims, filter_size, kernel_size, dropout, seq_len).to(device)
    trial.set_user_attr(key="best_model", value=model)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    criterion = torch.nn.BCELoss().to(device)
    counter = 0
    CNN_acc = []
    CNN_valacc = []
    train_loss_epoch = []
    train_acc_epoch = []
    val_loss_epoch = []
    val_acc_epoch = []
    time_epoch = []
    val_acc = 0
    model.train()
    for e in range(epochs):
        start_time = time.time()
        train_loss = []
        train_acc = []
        for inputs, labels in train_dl:
            inputs, labels = inputs.to(device), labels.to(device)
            model.zero_grad()
            logits, accuracy = model(inputs, labels)
            loss = criterion(logits,labels.float())
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 5)
            optimizer.step()
            train_loss.append(loss.item())
            train_acc.append(accuracy.item()*100/batch_size)
            if counter%100==0:
                print("Epoch: {}/{}".format(e,epochs),
                             "\tIteration: {}".format(counter),
                             "\t\tTrain Loss: {:.3f}".format(loss.item()),
                             "\tTrain Accuracy: {:.2f}".format(accuracy.item()*100/batch_size))
                CNN_acc.append(accuracy.item()*100/batch_size)
            counter += 1
        train_loss_epoch.append(np.round(np.mean(train_loss), 3))
        train_acc_epoch.append(np.round(np.mean(train_acc), 3))
        print("\tTrain Loss: {:.3f}".format(np.mean(train_loss)), "\tTrain Acc: {:.3f}".format(np.mean(train_acc)))

        with torch.no_grad():
            model.eval()
            val_acc = []
            val_loss = []
            for inputs, labels in val_dl:
                inputs_val, labels_val = inputs.to(device), labels.to(device)
                logits_val,accuracy_val = model(inputs_val,labels_val)
                loss_val = criterion(logits_val,labels_val.float())
                val_acc.append(accuracy_val.item()*100/batch_size)
                val_loss.append(loss_val.item())
            val_loss_epoch.append(np.round(np.mean(val_loss), 3))
            val_acc_epoch.append(np.round(np.mean(val_acc), 3))
            print("\t\tVal Loss: {:.3f}".format(np.mean(val_loss)), "\t\tVal Acc: {:.3f}".format(np.mean(val_acc)))
            CNN_valacc.append(np.mean(val_acc))
            model.train()
            val_acc = np.mean(val_acc)
        end_time = time.time()-start_time
        print("Time to train epoch: {0} s".format(end_time))
        time_epoch.append(np.round(end_time, 3))
    write_to_csv(trial.number,epochs, train_loss_epoch, train_acc_epoch, val_loss_epoch, val_acc_epoch, time_epoch)
    return np.mean(val_acc)


In [13]:
def callback(study, trial):
    if study.best_trial.number == trial.number:
        study.set_user_attr(key="best_model", value=trial.user_attrs["best_model"])

In [15]:
import optuna
from optuna.trial import TrialState
study = optuna.create_study(direction="maximize",pruner=optuna.pruners.HyperbandPruner(
        min_resource=1, max_resource=epochs, reduction_factor=3
    ),)
study.optimize(objective, n_trials=10, callbacks=[callback])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of complete trials: ", len(complete_trials))
print("Best trial:")
trial = study.best_trial
best_model=study.user_attrs["best_model"]
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-05-14 11:59:25,916] A new study created in memory with name: no-name-d1e7d92d-bb1b-4486-9f20-8a9ee940a95e


Epoch: 0/10 	Iteration: 0 		Train Loss: 0.670 	Train Accuracy: 62.00
Epoch: 0/10 	Iteration: 100 		Train Loss: 0.542 	Train Accuracy: 78.00
Epoch: 0/10 	Iteration: 200 		Train Loss: 0.558 	Train Accuracy: 70.00
Epoch: 0/10 	Iteration: 300 		Train Loss: 0.459 	Train Accuracy: 80.00
Epoch: 0/10 	Iteration: 400 		Train Loss: 0.333 	Train Accuracy: 84.00
Epoch: 0/10 	Iteration: 500 		Train Loss: 0.372 	Train Accuracy: 82.00
Epoch: 0/10 	Iteration: 600 		Train Loss: 0.232 	Train Accuracy: 94.00
Epoch: 0/10 	Iteration: 700 		Train Loss: 0.378 	Train Accuracy: 84.00
Epoch: 0/10 	Iteration: 800 		Train Loss: 0.368 	Train Accuracy: 90.00
Epoch: 0/10 	Iteration: 900 		Train Loss: 0.212 	Train Accuracy: 92.00
Epoch: 0/10 	Iteration: 1000 		Train Loss: 0.360 	Train Accuracy: 92.00
Epoch: 0/10 	Iteration: 1100 		Train Loss: 0.328 	Train Accuracy: 82.00
Epoch: 0/10 	Iteration: 1200 		Train Loss: 0.234 	Train Accuracy: 88.00
Epoch: 0/10 	Iteration: 1300 		Train Loss: 0.262 	Train Accuracy: 88.00
Epoc

Epoch: 6/10 	Iteration: 10500 		Train Loss: 0.330 	Train Accuracy: 86.00
Epoch: 6/10 	Iteration: 10600 		Train Loss: 0.226 	Train Accuracy: 88.00
Epoch: 6/10 	Iteration: 10700 		Train Loss: 0.184 	Train Accuracy: 94.00
Epoch: 6/10 	Iteration: 10800 		Train Loss: 0.492 	Train Accuracy: 80.00
Epoch: 6/10 	Iteration: 10900 		Train Loss: 0.198 	Train Accuracy: 88.00
Epoch: 6/10 	Iteration: 11000 		Train Loss: 0.402 	Train Accuracy: 80.00
Epoch: 6/10 	Iteration: 11100 		Train Loss: 0.264 	Train Accuracy: 86.00
Epoch: 6/10 	Iteration: 11200 		Train Loss: 0.323 	Train Accuracy: 90.00
Epoch: 6/10 	Iteration: 11300 		Train Loss: 0.188 	Train Accuracy: 96.00
Epoch: 6/10 	Iteration: 11400 		Train Loss: 0.165 	Train Accuracy: 94.00
Epoch: 6/10 	Iteration: 11500 		Train Loss: 0.425 	Train Accuracy: 80.00
Epoch: 6/10 	Iteration: 11600 		Train Loss: 0.172 	Train Accuracy: 94.00
	Train Loss: 0.258 	Train Acc: 89.164
		Val Loss: 0.398 		Val Acc: 87.800
Time to train epoch: 27.16994571685791 s
Epoch: 7/

[I 2022-05-14 12:03:58,631] Trial 0 finished with value: 87.83636363636364 and parameters: {'optimizer': 'RMSprop', 'lr': 0.0030178221030598457, 'filter_size': 56, 'dropout': 0.5373561859697323, 'embed_size': 73}. Best is trial 0 with value: 87.83636363636364.


		Val Loss: 0.405 		Val Acc: 87.836
Time to train epoch: 27.151519298553467 s
Epoch: 0/10 	Iteration: 0 		Train Loss: 0.591 	Train Accuracy: 80.00
Epoch: 0/10 	Iteration: 100 		Train Loss: 26.000 	Train Accuracy: 74.00
Epoch: 0/10 	Iteration: 200 		Train Loss: 30.000 	Train Accuracy: 70.00
Epoch: 0/10 	Iteration: 300 		Train Loss: 20.000 	Train Accuracy: 80.00
Epoch: 0/10 	Iteration: 400 		Train Loss: 22.000 	Train Accuracy: 78.00
Epoch: 0/10 	Iteration: 500 		Train Loss: 22.000 	Train Accuracy: 78.00
Epoch: 0/10 	Iteration: 600 		Train Loss: 24.000 	Train Accuracy: 76.00
Epoch: 0/10 	Iteration: 700 		Train Loss: 26.000 	Train Accuracy: 74.00
Epoch: 0/10 	Iteration: 800 		Train Loss: 20.000 	Train Accuracy: 80.00
Epoch: 0/10 	Iteration: 900 		Train Loss: 12.000 	Train Accuracy: 88.00
Epoch: 0/10 	Iteration: 1000 		Train Loss: 22.000 	Train Accuracy: 78.00
Epoch: 0/10 	Iteration: 1100 		Train Loss: 22.000 	Train Accuracy: 78.00
Epoch: 0/10 	Iteration: 1200 		Train Loss: 20.000 	Train Ac

...

[I 2022-05-14 12:35:11,796] Trial 8 finished with value: 81.58181818181818 and parameters: {'optimizer': 'Adam', 'lr': 2.907372953623839e-05, 'filter_size': 97, 'dropout': 0.38076923219844816, 'embed_size': 39}. Best is trial 0 with value: 87.83636363636364.


		Val Loss: 0.405 		Val Acc: 81.582
Time to train epoch: 10.649317502975464 s
Epoch: 0/10 	Iteration: 0 		Train Loss: 0.799 	Train Accuracy: 22.00
Epoch: 0/10 	Iteration: 100 		Train Loss: 26.000 	Train Accuracy: 74.00
Epoch: 0/10 	Iteration: 200 		Train Loss: 30.000 	Train Accuracy: 70.00
Epoch: 0/10 	Iteration: 300 		Train Loss: 20.000 	Train Accuracy: 80.00
Epoch: 0/10 	Iteration: 400 		Train Loss: 22.000 	Train Accuracy: 78.00
Epoch: 0/10 	Iteration: 500 		Train Loss: 22.000 	Train Accuracy: 78.00
Epoch: 0/10 	Iteration: 600 		Train Loss: 24.000 	Train Accuracy: 76.00
Epoch: 0/10 	Iteration: 700 		Train Loss: 26.000 	Train Accuracy: 74.00
Epoch: 0/10 	Iteration: 800 		Train Loss: 20.000 	Train Accuracy: 80.00
Epoch: 0/10 	Iteration: 900 		Train Loss: 12.000 	Train Accuracy: 88.00
Epoch: 0/10 	Iteration: 1000 		Train Loss: 22.000 	Train Accuracy: 78.00
Epoch: 0/10 	Iteration: 1100 		Train Loss: 22.000 	Train Accuracy: 78.00
Epoch: 0/10 	Iteration: 1200 		Train Loss: 20.000 	Train Ac

Epoch: 6/10 	Iteration: 10200 		Train Loss: 22.000 	Train Accuracy: 78.00
Epoch: 6/10 	Iteration: 10300 		Train Loss: 18.000 	Train Accuracy: 82.00
Epoch: 6/10 	Iteration: 10400 		Train Loss: 32.000 	Train Accuracy: 68.00
Epoch: 6/10 	Iteration: 10500 		Train Loss: 28.000 	Train Accuracy: 72.00
Epoch: 6/10 	Iteration: 10600 		Train Loss: 18.000 	Train Accuracy: 82.00
Epoch: 6/10 	Iteration: 10700 		Train Loss: 34.000 	Train Accuracy: 66.00
Epoch: 6/10 	Iteration: 10800 		Train Loss: 26.000 	Train Accuracy: 74.00
Epoch: 6/10 	Iteration: 10900 		Train Loss: 30.000 	Train Accuracy: 70.00
Epoch: 6/10 	Iteration: 11000 		Train Loss: 28.000 	Train Accuracy: 72.00
Epoch: 6/10 	Iteration: 11100 		Train Loss: 16.000 	Train Accuracy: 84.00
Epoch: 6/10 	Iteration: 11200 		Train Loss: 24.000 	Train Accuracy: 76.00
Epoch: 6/10 	Iteration: 11300 		Train Loss: 12.000 	Train Accuracy: 88.00
Epoch: 6/10 	Iteration: 11400 		Train Loss: 14.000 	Train Accuracy: 86.00
Epoch: 6/10 	Iteration: 11500 		Train 

[I 2022-05-14 12:40:10,156] Trial 9 finished with value: 75.6 and parameters: {'optimizer': 'Adam', 'lr': 0.057848460416103656, 'filter_size': 206, 'dropout': 0.4440156972958089, 'embed_size': 67}. Best is trial 0 with value: 87.83636363636364.


		Val Loss: 23.667 		Val Acc: 75.600
Time to train epoch: 29.85227656364441 s
Study statistics: 
  Number of finished trials:  10
  Number of complete trials:  10
Best trial:
  Value:  87.83636363636364
  Params: 
    optimizer: RMSprop
    lr: 0.0030178221030598457
    filter_size: 56
    dropout: 0.5373561859697323
    embed_size: 73


In [17]:
torch.save(best_model.state_dict(),"./cnn-amazon-hyperband/cnn_amazon_hyperband.pth")

In [20]:
criterion = torch.nn.BCELoss().to(device)
def test(model, test_dl, epochs):
    with torch.no_grad():
        model.eval()
        test_acc = []
        test_loss = []
        for inputs, labels in test_dl:
            input_test, labels_test = inputs.cuda(), labels.cuda()
            logits_test,accuracy_test = model(input_test,labels_test)
            loss_test = criterion(logits_test,labels_test.float())
            test_acc.append(accuracy_test.item()*100/batch_size)
            test_loss.append(loss_test.item())
        print("Test Loss: {:.3f}".format(np.mean(test_loss)), "\tTest Acc: {:.3f}".format(np.mean(test_acc)))
        #append_to_csv(epochs, np.round(np.mean(test_acc), 3))

In [21]:
test(best_model, test_dl, epochs)

Test Loss: 0.418 	Test Acc: 87.691
